This model got 0.86332

In [ ]:
# model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# ------------------------------------------------------------------------
#                       Squeeze-and-Excitation Block
# ------------------------------------------------------------------------
class SEBlock(nn.Module):
    """
    Implements the Squeeze-and-Excitation block which enhances feature maps by modeling interdependencies.
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)  # Reduce channel size
        self.fc2 = nn.Linear(channels // reduction, channels)  # Restore original size

    def forward(self, x):
        b, c, _, _ = x.size()  # Extract batch and channel dimensions
        # Squeeze: Global average pooling across spatial dimensions
        squeeze = x.view(b, c, -1).mean(dim=2)
        # Excitation: Fully connected layers with ReLU and Sigmoid
        excitation = F.relu(self.fc1(squeeze))
        excitation = torch.sigmoid(self.fc2(excitation)).view(b, c, 1, 1)
        # Scale the input by excitation values
        return x * excitation

# ------------------------------------------------------------------------
#                       Stochastic Depth Block
# ------------------------------------------------------------------------
class StochasticDepth(nn.Module):
    """
    Implements Stochastic Depth, randomly dropping residual branches during training.
    """
    def __init__(self, p: float = 0.1):
        super().__init__()
        self.p = p  # Probability of dropping the residual branch

    def forward(self, x, residual):
        if not self.training or self.p == 0.0:
            return x + residual  # No drop in evaluation mode or p=0
        if torch.rand(1).item() < self.p:  # Drop with probability p
            return x
        else:
            return x + residual  # Keep residual branch

# ------------------------------------------------------------------------
#       PreAct Residual Block with SE + StochasticDepth
# ------------------------------------------------------------------------
class PreActBlock(nn.Module):
    """
    Pre-activation residual block with SE and Stochastic Depth integration.
    """
    def __init__(self, in_planes, out_planes, stride=1, drop_prob=0.0):
        super().__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)  # Pre-activation batch norm
        self.se = SEBlock(in_planes)           # SE block for channel attention
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)

        # Optional shortcut for downsampling or channel size change
        self.shortcut = None
        if stride != 1 or in_planes != out_planes:
            self.shortcut = nn.Conv2d(in_planes, out_planes, kernel_size=1,
                                      stride=stride, padding=0, bias=False)

        # Stochastic Depth integration
        self.sd = StochasticDepth(p=drop_prob)

    def forward(self, x):
        out = F.relu(self.bn1(x))    # Pre-activation
        out = self.se(out)           # SE block applied to pre-activated features
        shortcut = x if self.shortcut is None else self.shortcut(out)
        out = self.conv1(out)
        out = F.relu(self.bn2(out))
        out = self.conv2(out)
        return self.sd(shortcut, out)  # Apply stochastic depth to residual connection

# ------------------------------------------------------------------------
#       Modified LightResNet for CIFAR-10 with ~4.4M parameters
# ------------------------------------------------------------------------
class LightResNet18_v2(nn.Module):
    """
    A lightweight ResNet-18 variant designed for CIFAR-10 with reduced parameter count.
    """
    def __init__(self, num_classes=10, drop_prob=0.1):
        super().__init__()
        self.in_planes = 40  # Base channel width increased to 40
        self.conv1 = nn.Conv2d(3, 40, kernel_size=3, stride=1,
                               padding=1, bias=False)

        # Create 4 layers with increasing channel sizes: 40 -> 80 -> 160 -> 320
        self.layer1 = self._make_layer(40, 2, stride=1, base_p=drop_prob * 1/4)
        self.layer2 = self._make_layer(80, 2, stride=2, base_p=drop_prob * 2/4)
        self.layer3 = self._make_layer(160, 2, stride=2, base_p=drop_prob * 3/4)
        self.layer4 = self._make_layer(320, 2, stride=2, base_p=drop_prob * 4/4)

        self.bn = nn.BatchNorm2d(320)  # Final batch normalization
        self.linear = nn.Linear(320, num_classes)  # Fully connected output layer

        # Weight initialization for improved convergence
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, out_planes, blocks, stride, base_p):
        """Constructs a series of residual blocks with increasing dropout probability."""
        strides = [stride] + [1] * (blocks - 1)
        layers = []
        for i in range(blocks):
            block_p = base_p * (i + 1) / blocks  # Progressive drop probability
            layers.append(PreActBlock(self.in_planes, out_planes,
                                      stride=strides[i],
                                      drop_prob=block_p))
            self.in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.relu(self.bn(out))  # Final activation
        out = F.adaptive_avg_pool2d(out, 1).view(out.size(0), -1)  # Global average pooling
        out = self.linear(out)      # Fully connected output layer
        return out

# Quick parameter check
if __name__ == "__main__":
    model = LightResNet18_v2(num_classes=10, drop_prob=0.1)
    x = torch.randn(2, 3, 32, 32)  # Sample CIFAR-10 batch input
    y = model(x)
    print("Output shape:", y.shape)
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {total_params:,} (~{total_params/1e6:.2f}M)")


Output shape: torch.Size([2, 10])
Total trainable parameters: 4,389,826 (~4.39M)


In [ ]:
# train.py
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import datasets
import numpy as np
import pickle
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#from model import LightResNet18  # Assuming this is a custom model definition
from tqdm import tqdm

# Custom collate function for MixUp/CutMix data augmentation
# Randomly selects between MixUp and CutMix for each batch
def rand_bbox(size, lam):
    """Generate random bounding box coordinates for CutMix"""
    W = size[3]
    H = size[2]
    cut_rat = np.sqrt(1. - lam)  # Cut ratio based on lambda
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    # Center coordinates
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    # Calculate bounding box coordinates with clipping
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def mixup_data(x, y, alpha=1.0):
    """Applies MixUp augmentation to a batch of data"""
    lam = np.random.beta(alpha, alpha)  # Mix ratio from Beta distribution
    batch_size = x.size(0)
    index = torch.randperm(batch_size)  # Shuffled indices
    mixed_x = lam * x + (1 - lam) * x[index, :]  # Mix images
    y_a, y_b = y, y[index]  # Mix labels
    return mixed_x, y_a, y_b, lam

def cutmix_data(x, y, alpha=1.0):
    """Applies CutMix augmentation to a batch of data"""
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size(0)
    index = torch.randperm(batch_size)

    # Generate bounding box coordinates
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)

    # Replace selected region with another image's region
    x[:, :, bby1:bby2, bbx1:bbx2] = x[index, :, bby1:bby2, bbx1:bbx2]

    # Adjust lambda based on the area ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size(-1) * x.size(-2)))
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

def mixup_cutmix_collate(batch, alpha=1.0, p=0.5):
    """Custom collate function that applies either MixUp or CutMix randomly"""
    # Unpack the batch into images and labels
    images, labels = list(zip(*batch))
    images = torch.stack(images, 0)
    labels = torch.tensor(labels, dtype=torch.long)

    # Randomly choose between MixUp and CutMix
    if np.random.rand() < p:
        return mixup_data(images, labels, alpha)
    else:
        return cutmix_data(images, labels, alpha)

def unpickle(file):
    """Unpickle CIFAR-10 batch files"""
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARDataset(Dataset):
    """Custom Dataset class for CIFAR-10 with optional transforms"""
    def __init__(self, data, labels, transform=None):
        self.data = data.reshape(-1, 3, 32, 32).astype("float32") / 255.0
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Convert numpy array to PIL Image
        img = self.data[idx]*255
        img = img.astype("uint8")
        img = Image.fromarray(img.transpose(1,2,0))  # Change from CHW to HWC
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def load_cifar10_batches(root_dir):
    """Load and concatenate all CIFAR-10 training batches"""
    data_list, labels_list = [], []
    for i in range(1, 6):
        batch_file = f"{root_dir}/data_batch_{i}"
        batch = unpickle(batch_file)
        data_list.append(batch[b'data'])
        labels_list.extend(batch[b'labels'])
    X = np.concatenate(data_list, axis=0)
    y = np.array(labels_list)
    return X, y

def train():
    """Main training function"""
    # Set device (GPU if available)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Data loading and preprocessing
    # Note: The original code uses torchvision's built-in CIFAR-10 dataset
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandAugment(),  # Random augmentation policy
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),  # CIFAR-10 mean
        (0.2470, 0.2435, 0.2616))  # CIFAR-10 std
    ])
    transform_val = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
        (0.2470, 0.2435, 0.2616))
    ])

    # Create datasets and dataloaders
    train_dataset = datasets.CIFAR10(root='./data', train=True,
                                 download=True, transform=transform_train)
    val_dataset = datasets.CIFAR10(root='./data', train=False,
                                download=True, transform=transform_val)

    # Create DataLoader with custom collate function for augmentation mixing
    train_loader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, num_workers=4,
                              collate_fn=lambda b: mixup_cutmix_collate(b, alpha=1.0, p=0.5))
    val_loader = DataLoader(val_dataset, batch_size=128,
                              shuffle=False, num_workers=4)

    # Initialize model
    model = LightResNet18_v2(num_classes=10, drop_prob=0.1).to(device)

    # Training setup
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  # Regularization
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,
                        nesterov=True, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

    best_acc = 0.0
    num_epochs = 300

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_approx = 0.0
        total_approx = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)
        for images, label_info in train_pbar:
            # Unpack mixup/cutmix labels and lambda
            y_a, y_b, lam, method = label_info
            images = images.to(device)
            y_a = y_a.to(device)
            y_b = y_b.to(device)

            optimizer.zero_grad()
            outputs = model(images)

            # Calculate mixed loss
            loss = lam * criterion(outputs, y_a) + (1 - lam) * criterion(outputs, y_b)
            loss.backward()
            optimizer.step()

            # Track metrics
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_approx += (preds == y_a).sum().item() * lam + \
                            (preds == y_b).sum().item() * (1 - lam)
            total_approx += images.size(0)

            train_pbar.set_postfix(loss=f"{loss.item():.3f}")

        # Update learning rate
        scheduler.step()
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100.0 * correct_approx / total_approx

        # Validation phase
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, pred = torch.max(outputs, 1)
                val_correct += (pred == labels).sum().item()
                val_total += labels.size(0)
        val_acc = 100.0 * val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_loss:.4f}, Approx Train Acc: {epoch_acc:.2f}% | "
              f"Val Acc: {val_acc:.2f}%")

        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"   [*] Saved new best model: Val Acc = {val_acc:.2f}%")

    print(f"Training complete. Best validation accuracy: {best_acc:.2f}%")
    print("Best model saved as best_model.pth")

if __name__ == "__main__":
    train()

Using device: cuda


100%|██████████| 170M/170M [00:18<00:00, 9.17MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/300 | Train Loss: 2.0815, Approx Train Acc: 26.68% | Val Acc: 47.50%
   [*] Saved new best model: Val Acc = 47.50%


...

Epoch 296/300 | Train Loss: 1.2306, Approx Train Acc: 69.55% | Val Acc: 96.01%


Epoch 297/300 | Train Loss: 1.2342, Approx Train Acc: 69.25% | Val Acc: 95.88%


Epoch 298/300 | Train Loss: 1.2306, Approx Train Acc: 69.69% | Val Acc: 96.11%
   [*] Saved new best model: Val Acc = 96.11%


Epoch 299/300 | Train Loss: 1.2150, Approx Train Acc: 70.00% | Val Acc: 95.87%


Epoch 300/300 | Train Loss: 1.2110, Approx Train Acc: 70.83% | Val Acc: 95.75%
Training complete. Best validation accuracy: 96.11%
Best model saved as best_model.pth


In [ ]:
# inference.py
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle
import numpy as np
from torch.utils.data import DataLoader, Dataset
#from model import LightResNet18
import torch.nn.functional as F

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: shape (N, 32, 32, 3)
        ids: array/list of image IDs
        transform: transforms to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].astype("uint8")  # ensure 0..255
        img_id = self.ids[idx]
        if self.transform:
            img = self.transform(img)
        return img, img_id

def inference():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")

    # 1) Load the best model
    model = LightResNet18_v2(num_classes=10, drop_prob=0.1).to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()

    # 2) Load custom test set .pkl
    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    print("Keys in test_dict:", test_dict.keys())

    # Reshape if it's (N, 3072). If it's already (N, 32,32,3), remove reshape
    test_images = test_dict[b'data'].reshape(-1, 32, 32, 3)
    test_ids = [str(i) for i in range(len(test_images))]

    # 3) Define test transforms
    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2470, 0.2435, 0.2616))
    ])

    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    # 4) Inference with 3-pass TTA
    predictions = []
    image_ids = []
    with torch.no_grad():
        for imgs, ids in test_loader:
            imgs = imgs.to(device)

            # Pass 1: Normal
            out_normal = model(imgs)
            probs_normal = F.softmax(out_normal, dim=1)

            # Pass 2: Horizontal flip
            imgs_flipped_h = torch.flip(imgs, dims=[3])  # flip W dimension
            out_flipped_h = model(imgs_flipped_h)
            probs_flipped_h = F.softmax(out_flipped_h, dim=1)

            # Pass 3: Vertical flip
            imgs_flipped_v = torch.flip(imgs, dims=[2])  # flip H dimension
            out_flipped_v = model(imgs_flipped_v)
            probs_flipped_v = F.softmax(out_flipped_v, dim=1)

            # Average probabilities
            final_probs = (probs_normal + probs_flipped_h + probs_flipped_v) / 3.0
            _, predicted = torch.max(final_probs, 1)

            predictions.extend(predicted.cpu().numpy().tolist())
            image_ids.extend(ids)

    # 5) Save submission
    submission_df = pd.DataFrame({"ID": image_ids, "Labels": predictions})
    submission_df.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

if __name__ == "__main__":
    inference()


Using device: cuda
Keys in test_dict: dict_keys([b'data', b'ids'])


<ipython-input-7-9d43b5f99a1b>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))
/usr/local/lib/pyth

Submission file saved as submission.csv


In [ ]:
import pandas as pd

# Load and inspect submission file
submission_df = pd.read_csv("submission.csv")
print(submission_df.head())  # Show first few rows
print(submission_df["Labels"].value_counts())  # Show label distribution


   ID  Labels
0   0       6
1   1       1
2   2       8
3   3       6
4   4       9
Labels
3    1194
1    1022
5    1022
8    1012
9     999
2     967
7     965
0     941
6     939
4     939
Name: count, dtype: int64


ID  Labels

0   0       6

1   1       1

2   2       8

3   3       6

4   4       9

Labels

3    1119

1    1066

8    1053

5    1040

7    1006

2     983

9     980

4     949

6     918

0     886

Name: count, dtype: int64
